In [36]:
import os
thisdir="C:\\Users\\adit4\\Desktop\\UC\\test\\QrespData\\a317eb6e5dc0baf51572a53b6334ade6731b6979405b37911b701e10beebecde"
import requests
headers = {"Content-Type": "application/json"}
ACCESS_TOKEN = "YgQHzUqXB4uPZUyAJDKjzmq30LpY1lfssfziZAppotkVPJEtxQpXEuO2XQLk"
response = requests.post('https://sandbox.zenodo.org/api/deposit/depositions',params={'access_token': ACCESS_TOKEN}, json={},headers=headers)
print(response.json())
deposition_id = response.json()['id']
for r, d, f in os.walk(thisdir):
    for metadata_file in f:
        if metadata_file.lower().endswith((".jpg",".jpeg",".png",".gif",".tiff",".pdf",".bmp",".ico",".svg")):
            data = {'filename': str(metadata_file)}
            filepath = os.path.join(r,metadata_file)
            files = {'file': open(str(filepath), 'rb')}
            resp = requests.post('https://sandbox.zenodo.org/api/deposit/depositions/%s/files' % deposition_id,params={'access_token': ACCESS_TOKEN}, data=data,files=files)
            resp.json()

{'conceptrecid': '317162', 'created': '2019-07-19T06:31:53.160868+00:00', 'files': [], 'id': 317163, 'links': {'bucket': 'https://sandbox.zenodo.org/api/files/3648f19f-e25e-4205-bf31-68c2559eedba', 'discard': 'https://sandbox.zenodo.org/api/deposit/depositions/317163/actions/discard', 'edit': 'https://sandbox.zenodo.org/api/deposit/depositions/317163/actions/edit', 'files': 'https://sandbox.zenodo.org/api/deposit/depositions/317163/files', 'html': 'https://sandbox.zenodo.org/deposit/317163', 'latest_draft': 'https://sandbox.zenodo.org/api/deposit/depositions/317163', 'latest_draft_html': 'https://sandbox.zenodo.org/deposit/317163', 'publish': 'https://sandbox.zenodo.org/api/deposit/depositions/317163/actions/publish', 'self': 'https://sandbox.zenodo.org/api/deposit/depositions/317163'}, 'metadata': {'prereserve_doi': {'doi': '10.5072/zenodo.317163', 'recid': 317163}}, 'modified': '2019-07-19T06:31:53.160877+00:00', 'owner': 28783, 'record_id': 317163, 'state': 'unsubmitted', 'submitted

In [38]:
import json

r = requests.put('https://sandbox.zenodo.org/api/deposit/depositions/%s' % deposition_id,params={'access_token': ACCESS_TOKEN}, data=json.dumps(data),
                 headers=headers)
r.status_code

200

In [100]:
import shutil
import requests
import os
import json
import sys
class UploadToZenodo:
    """ Upload Files to Zenodo
    """
    def __init__(self,ACCESS_TOKEN,metadata):
        self.headers = {"Content-Type": "application/json"}
        self.params = {'access_token': ACCESS_TOKEN}
        self.base_url = "https://sandbox.zenodo.org"
        self.deposition_id = self.generateDepositionId()
        self.acceptedExtentions = (".jpg",".jpeg",".png",".gif",".tiff",".pdf",".bmp",".ico",".svg")
        self.metadata = metadata
    
    def generateDepositionId(self):
        """ Generates Deposition Id needed for identifying paper
        """
        response = requests.post('{base_url}/api/deposit/depositions'.format(base_url=self.base_url),
                                 params=self.params, 
                                 json={},headers=self.headers)
        if response.status_code >= 400 :
            sys.exit(response.json())
        return response.json()['id']
    
    def uploadImagesToZenodo(self,path):
        """ Extracts Images from path and uploads to zenodo
            :param: path Directory or path to directory to upload data from
        """
        for r, d, f in os.walk(path):
            for file in f:
                if file.lower().endswith(self.acceptedExtentions):
                    data = {'filename': str(file)}
                    filepath = os.path.join(r,file)
                    files = {'file': open(str(filepath), 'rb')}
                    resp = requests.post('{base_url}/api/deposit/depositions/{deposition_id}/files'.format(base_url=self.base_url,deposition_id=self.deposition_id),
                                         params=self.params, 
                                         data=data,files=files)
                    if resp.status_code >= 400 :
                        sys.exit(resp.json())
        print("Uploaded images to deposition id ",self.deposition_id)
    
    def uploadZipFileToZenodo(self,path):
        """ Converts folder to a zip file and uploads to zenodo
            :param: path Directory or path to directory to upload data from
        """
        filename = os.path.basename(path)
        archive_from = os.path.dirname(path)
        archive_to = os.path.basename(path.strip(os.sep))
        shutil.make_archive(filename, "zip", archive_from, archive_to)
        data = {'filename': str(filename)+".zip"}
        files = {'file': open(str(filename)+".zip", 'rb')}
        resp = requests.post('{base_url}/api/deposit/depositions/{deposition_id}/files'.format(base_url=self.base_url,deposition_id=self.deposition_id),
                                         params=self.params, 
                                         data=data,files=files)
        if resp.status_code >= 400 :
            sys.exit(resp.json())
        print("\n Uploaded directory as a zip file ",self.deposition_id)

    def uploadMetadaFileToZenodo(self):
        """ Adds metadata to the deposition
        """
        resp = requests.put('{base_url}/api/deposit/depositions/{deposition_id}'.format(base_url=self.base_url,deposition_id=self.deposition_id),
                             params=self.params, 
                             data=json.dumps(self.metadata),
                             headers=self.headers)
        if resp.status_code >= 400 :
            sys.exit(resp.json())
        print("\n Uploaded metadata file")
        
    def publishProjectToZenodo(self):
        """ Publishes metadata to zenodo
        """
        resp = requests.post('{base_url}/api/deposit/depositions/{deposition_id}/actions/publish'.format(base_url=self.base_url,deposition_id=self.deposition_id),
                             params=self.params,  
                            headers=self.headers)
        if resp.status_code >= 400 :
            sys.exit(resp.json())
        return resp.json()

In [101]:
data = {
    'metadata': {
        'title': 'My fourth upload',
        'upload_type': 'dataset',
        'description': 'This is my first upload',
        'creators': [{'name': 'Aditya',
                      'affiliation': 'Qresp'}]
    }
}

ACCESS_TOKEN = "YgQHzUqXB4uPZUyAJDKjzmq30LpY1lfssfziZAppotkVPJEtxQpXEuO2XQLk"

In [102]:
uploadZ = UploadToZenodo(ACCESS_TOKEN,data)
    

In [103]:
uploadZ.uploadImagesToZenodo("C:\\Users\\adit4\\Desktop\\UC\\paper")

Uploaded images to deposition id  326415


In [104]:
print(uploadZ.deposition_id)

326415


In [105]:
uploadZ.uploadZipFileToZenodo("C:\\Users\\adit4\\Desktop\\UC\\paper")


 Uploaded directory as a zip file  326415


In [106]:
uploadZ.uploadMetadaFileToZenodo()


 Uploaded metadata file


In [107]:
record = uploadZ.publishProjectToZenodo()
print(record)

{'conceptdoi': '10.5072/zenodo.326414', 'conceptrecid': '326414', 'created': '2019-07-22T22:33:42.880268+00:00', 'doi': '10.5072/zenodo.326415', 'doi_url': 'https://doi.org/10.5072/zenodo.326415', 'files': [{'checksum': 'e7b51943d6edde41333c98552044aca1', 'filename': 'Qresp_1.jpg', 'filesize': 101370, 'id': 'e409582c-ac5c-45a2-8676-2f7bc76c304b', 'links': {'download': 'https://sandbox.zenodo.org/api/files/0065b580-3b68-4cab-9331-7897b3a08114/Qresp_1.jpg', 'self': 'https://sandbox.zenodo.org/api/deposit/depositions/317114/files/e409582c-ac5c-45a2-8676-2f7bc76c304b'}}, {'checksum': 'ce749a943a6103defacddc3b763c3e7e', 'filename': 'Qresp_1.png', 'filesize': 128328, 'id': '14aee0e2-c8fc-42cd-ab0f-f37b96b3ed85', 'links': {'download': 'https://sandbox.zenodo.org/api/files/0065b580-3b68-4cab-9331-7897b3a08114/Qresp_1.png', 'self': 'https://sandbox.zenodo.org/api/deposit/depositions/317114/files/14aee0e2-c8fc-42cd-ab0f-f37b96b3ed85'}}, {'checksum': '165d4e78012de2476374cdfe9acfd737', 'filename'

In [108]:
'{base_url}/api/deposit/depositions/{deposition_id}'.format(base_url=base_url,deposition_id=deposition_id)

'https://sandbox.zenodo.org/api/deposit/depositions/326390'

In [109]:
%pprint(record)

Pretty printing has been turned ON


In [110]:
record['doi_url']

'https://doi.org/10.5072/zenodo.326415'

In [111]:
record['links']['latest_html']

'https://sandbox.zenodo.org/record/326415'

In [58]:
record['doi_url']

'https://doi.org/10.5072/zenodo.326407'

In [85]:
os.path.dirname("C:\\Users\\adit4\\Desktop\\UC\\paper")

'C:\\Users\\adit4\\Desktop\\UC'

In [92]:
import os, shutil
def make_archive(source):
        base = os.path.basename(source)
        archive_from = os.path.dirname(source)
        archive_to = os.path.basename(source.strip(os.sep))
        print(source, archive_from, archive_to)
        shutil.make_archive(base, "zip", archive_from, archive_to)
        #shutil.move('%s.%s'%(name,format), destination)

In [93]:
make_archive("C:\\Users\\adit4\\Desktop\\UC\\paper")

C:\Users\adit4\Desktop\UC\paper C:\Users\adit4\Desktop\UC paper


In [17]:
import os
for r, d, f in os.walk("C:\\Users\\adit4\\Desktop\\UC\\paper"):
    for file in f:
        if file.lower().endswith((".jpg",".jpeg",".png",".gif",".tiff",".pdf",".bmp",".ico",".svg")):
            os.rename(file,os.path.split(r)[1]+"__"+file)
            #print(pname)
            filepath = os.path.join(r,file)
            data = {'filename': os.path.split(r)[1]+"\\"+str(file)}
            #files = {'file': open(str(filepath), 'rb')}
            print(filepath,"\n >",r,"\n >",d,"\n >",file,"\n >",data)

FileNotFoundError: [WinError 2] The system cannot find the file specified: 'Qresp_1.jpg' -> 'paper__Qresp_1.jpg'